In [1]:
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd # type: ignore

In [2]:
from utils import get_dataset, get_model_by_tag, get_prompt_template
from LLMAnnotator import LLMAnnotator

In [3]:
os.environ['OPENAI_API_KEY_CLARIN']=os.getenv("OPENAI_API_KEY_CLARIN")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

token = os.getenv("OPENAI_API_KEY_CLARIN")

In [4]:
model_tag = 'gpt4omini'  
dataset_tag = 'ag_news'
prompt = 'cot_random_samples'
temp = 0.3
number_of_experimet = 2
selected_samples =  f'{model_tag}_random42_80'

In [5]:
model = get_model_by_tag(model_tag, token, temp)  
dataset_for_annotation = get_dataset(dataset_tag=dataset_tag)
prompt_txt = get_prompt_template(dataset_tag, prompt)
examples_for_prompt = pd.read_csv(f'./selected_samples/{dataset_tag}/{selected_samples}.csv')

output_path = f'./results/{prompt}_temp{temp}/{model_tag}/{dataset_tag}_{model_tag}_{prompt}_{selected_samples}_temp{temp}_exp{number_of_experimet}.csv'
os.makedirs(f'./results/{prompt}_temp{temp}/{model_tag}', exist_ok=True)


In [6]:
dataset_for_annotation.columns

Index(['label', 'text'], dtype='object')

In [7]:
print(model, dataset_for_annotation, prompt_txt, examples_for_prompt, output_path)

GPT4oMiniLLM
Params: {}         label                                               text
0    Business  Coke is it: BHP coal to double BHP Billiton pl...
1    Business  BIG HIKE IN ENERGY BILLS The company says the ...
2    Business  Officials Blame Contractors in Tunnel Leak The...
3    Business  Before the Bell: Krispy Kreme Down 11 Pct Shar...
4      Sports  Medical Examiner Finds No Injuries on Thanou-S...
..        ...                                                ...
995     World  Indian politician sent to prison Top Hindu nat...
996    Sports  Ravens - Bills: Game Notes COMPOSED: With Lewi...
997  Sci/Tech  Dell touts in-home services Dell offers to ins...
998    Sports  Lately, it's been fast times World Series anal...
999     World  Venezuela Raises Taxes on Orinoco Deals &lt;p&...

[1000 rows x 2 columns] Task: Read the following text and determine the most appropriate category based on its content and the examples below.

Categories:
- World
- Sports
- Business
- Sci/Tech


In [8]:
text_col = "text"
label_col = "label"
output_col = "output"

In [9]:
annotator = LLMAnnotator(
    model=model,
    dataset=dataset_for_annotation[:100],
    examples_for_prompt=examples_for_prompt,
    prompt_template=prompt_txt,
    column_text=text_col,
    column_label=label_col,
    column_output=output_col
)


In [10]:
df = annotator.get_results()

/home/julita/Desktop/anote-skrpyty 19.04/LLMAnnotator.py:77: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(llm=self.model, prompt=self.prompt)


Nr: 0 Predicted label: To categorize the provided text, we need to analyze its content. The text discusses BHP Billiton's plans to significantly increase its coking coal production to meet rising demand, particularly from China. This involves financial figures and industry-specific terms related to production and resources, which are characteristic of business activities.

The key elements in the text include:
- The mention of BHP Billiton, a major corporation.
- Discussion of financial investments (US$1.5-2 billion).
- Reference to market demand (China's demand for carbon steel-making materials).
- Focus on production and expansion in the coal industry.

Given these points, the text clearly falls under the category of "Business."

Final answer: [Business]
Nr: 1 Predicted label: To categorize the provided text, let's analyze its content step by step.

1. **Content Overview**: The text discusses a significant increase in energy bills, attributing this rise to the record price of wholesa

In [11]:
annotator.save_results(output_path=output_path)

Results saved successfully to ./results/cot_random_samples_temp0.3/gpt4omini/ag_news_gpt4omini_cot_random_samples_gpt4omini_random42_80_temp0.3_exp2.csv


In [12]:
df = pd.read_csv(output_path)
df

,text,output,logprobs,top_logprobs,original_label
0,Coke is it: BHP coal to double BHP Billiton pl...,"To categorize the provided text, we need to an...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Business
1,BIG HIKE IN ENERGY BILLS The company says the ...,"To categorize the provided text, let's analyze...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Business
2,Officials Blame Contractors in Tunnel Leak The...,"To categorize the provided text, we need to an...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Business
3,Before the Bell: Krispy Kreme Down 11 Pct Shar...,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Business
4,Medical Examiner Finds No Injuries on Thanou-S...,The text discusses a medical examination of Gr...,"{'content': [{'token': 'The', 'bytes': [84, 10...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Sports
...,...,...,...,...,...
95,"McNair, Brown Questionable Injuries again kept...",To determine the appropriate category for the ...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Sports
96,Briefly: Piracy crackdown yields \$2.2 million...,"To categorize the provided text, we need to an...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Sci/Tech
97,Sharon #39;s bold move in Gaza Tuesday #39;s v...,"To categorize the provided text, we need to an...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",World
98,Darfur Mediators Demand Respect of Cease-fire ...,"To categorize the provided text, we need to an...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",World


In [13]:
import pandas as pd
import re

def extract_bracketed_word(df, read_col, write_col):
    """
    Wyodrębnia pierwszy wyraz objęty nawiasami [] z kolumny `read_col`
    i zapisuje go do kolumny `write_col`.
    
    :param df: DataFrame wejściowy
    :param read_col: Nazwa kolumny z której odczytywany jest tekst
    :param write_col: Nazwa kolumny do której zapisywany będzie wynik
    :return: DataFrame z nową kolumną
    """
    def extract_first_bracketed(text):
        match = re.search(r'\[([^\[\]]+?)\]', str(text))
        return match.group(1) if match else None
    
    df[write_col] = df[read_col].apply(extract_first_bracketed)
    return df


In [14]:
import pandas as pd

def validate_text_column(path, df_reference):
    # Wczytaj DataFrame z pliku CSV
    df = pd.read_csv(path)
    
    # Pobierz unikalne wartości z kolumny 'text' z df_reference
    reference_texts = set(df_reference['text'].unique())
    
    # Dodaj kolumnę 'is_valid'
    df['was_in_selected_samples'] = df['text'].isin(reference_texts)
    df = extract_bracketed_word(df, output_col, 'extracted_label')
    df.to_csv(path, index=False)
    
    return df


In [15]:
df_new = validate_text_column(output_path, examples_for_prompt)
df_new

,text,output,logprobs,top_logprobs,original_label,was_in_selected_samples,extracted_label
0,Coke is it: BHP coal to double BHP Billiton pl...,"To categorize the provided text, we need to an...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Business,False,Business
1,BIG HIKE IN ENERGY BILLS The company says the ...,"To categorize the provided text, let's analyze...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Business,False,Business
2,Officials Blame Contractors in Tunnel Leak The...,"To categorize the provided text, we need to an...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Business,False,Business
3,Before the Bell: Krispy Kreme Down 11 Pct Shar...,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Business,False,Business
4,Medical Examiner Finds No Injuries on Thanou-S...,The text discusses a medical examination of Gr...,"{'content': [{'token': 'The', 'bytes': [84, 10...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Sports,False,Sports
...,...,...,...,...,...,...,...
95,"McNair, Brown Questionable Injuries again kept...",To determine the appropriate category for the ...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Sports,False,Sports
96,Briefly: Piracy crackdown yields \$2.2 million...,"To categorize the provided text, we need to an...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Sci/Tech,False,Business
97,Sharon #39;s bold move in Gaza Tuesday #39;s v...,"To categorize the provided text, we need to an...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",World,False,World
98,Darfur Mediators Demand Respect of Cease-fire ...,"To categorize the provided text, we need to an...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",World,False,World
